## 第1步 导入相关库

In [3]:
from tqdm import tqdm
import numpy as np
import torch
from torch.utils.data import DataLoader
import pdal
import laspy

from model import RandLANet
import os
from predict import PredictCloudDataset, ActiveLearningSampler

## 第2步 将PredictCloudDataset模块嵌入
PredictCloudDataset 将原始点云载入内存并构建kdtree

## 第3步 将ActiveLearningSampler模块嵌入
ActiveLearningSampler利用计算ossibility机制实现对输入点云的邻近采样

## 第4步 配置参数文件

### 4.1 配置network参数字典

In [4]:
network_dict = {
    "num_neighbors": 16,  # 在进行位置编码时搜索的邻域点个数
    "decimation": 4,  # 在encoding阶段下采样的比例，每次下采样采样点云个数除以4
}

### 4.2 配置data参数字典

In [5]:
data_dict = {
    "num_classes": 2,  # 点云标注的类别数
    "grid_size": 1.0,  # 降采样格网尺寸，1.0代表1m * 1m的范围内取一个点
    "HasIntensity": True,  # 数据集是否有intensity属性，有也可以填False，表示不使用，默认使用
    "HasRGB": False  # 数据集是否有RGB属性，有也可以填False，表示不使用
}

### 4.3 配置预测参数字典

In [6]:
predict_dict = {
    "input_path": 'D:/Data/FEIMA_DATA/lidar30/1/merged.las',  # 输入文件路径
    "save_path": 'D:/Data/FEIMA_DATA/lidar30/1/',  # 输出预测结果目录
    "epoch": 10,  # 预测总轮数
    "gpu": 0,  #  gpu
    "batch_size": 32,  # 预测的批次大小，即包含的点云份数，可以比train的大
    "step_size": 16,  # 这一个batch在一个epoch里预测多少次
    "num_points_per_step": 10000,  # 每一份点云有多少个点，与train对应
    "num_workers": 1, # 设置多少线程读取内存里的点云组织成batch
    "smooth": 0.95,  # 对预测标签进行平滑。0 不平滑， 0.99 更大的平滑
    "checkpoints": 'C:/Users/xhy/Desktop/RandLA-Net-pytorch-master/outputs/ckpts/2025-7-31/checkpoint_114_ACC_0.92_MIOU_0.77.pth',  # 加载网络模型的路径
    "PredictHasRGB": True,  # 预测的数据集是否有RGB，有就在输出的时候给点云赋色
    "PredictHasIntensity": True  # 预测的数据集是否有Intensity, 默认有
}

### 4.5 合并字典

In [7]:
merged_dict = network_dict | predict_dict | data_dict

## 第5步 实例化data loader
与train文件的evaluate部分一样，循环将dataloader里面的数据送入载入的模型进行预测

In [8]:
dataset = PredictCloudDataset(merged_dict)
device = torch.device('cuda')
torch.cuda.set_device(int(merged_dict["gpu"]))
cloud_data = dataset.inputs
pos = dataset.raw_pos
point_r = cloud_data['r']
point_g = cloud_data['g']
point_b = cloud_data['b']

sampler = ActiveLearningSampler(
    config=merged_dict,
    dataset=dataset,
    batch_size=merged_dict["batch_size"],
    step_size=merged_dict["step_size"]
)
dataloader = DataLoader(
    sampler,
    batch_size=merged_dict["batch_size"],
    num_workers=merged_dict["num_workers"],
    pin_memory=True,
    drop_last=True
)
print(dataloader)

## 第6步 初始化model

In [9]:
num_classes = merged_dict["num_classes"]
d_in = 3
if merged_dict["HasRGB"]:
    d_in += 3
if merged_dict["HasIntensity"]:
    d_in += 1

model = RandLANet(
    d_in,
    num_classes,
    num_neighbors=merged_dict["num_neighbors"],
    decimation=merged_dict["decimation"],
    device=device
).to(device)
print(model)

RandLANet(
  (fc_start): Linear(in_features=4, out_features=8, bias=True)
  (bn_start): Sequential(
    (0): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (encoder): ModuleList(
    (0): LocalFeatureAggregation(
      (mlp1): SharedMLP(
        (adapter): Identity()
        (conv): Conv1d(8, 8, kernel_size=(1,), stride=(1,))
        (bn): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): LeakyReLU(negative_slope=0.2)
      )
      (lse1): LocalSpatialEncoding(
        (mlp): SharedMLP(
          (adapter): Identity()
          (conv): Conv1d(10, 8, kernel_size=(1,), stride=(1,))
          (bn): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation): ReLU()
        )
      )
      (pool1): AttentivePooling(
        (linear): Linear(in_features=16, out_features=16, bias=True)
        (softmax): Softmax(dim=-2)
    

## 第7步 载入训练好的网络模型

In [10]:
# 加载完整检查点
checkpoint = torch.load(merged_dict["checkpoints"], map_location="cuda", weights_only=False)

# 直接提取 model_state_dict
model_state_dict = checkpoint['model_state_dict']

# 加载到模型中（strict=True 推荐用于严格匹配）
model.load_state_dict(model_state_dict, strict=True)

model.eval()

num_points = len(dataset.inputs)
all_preds = torch.zeros((num_points, merged_dict["num_classes"])).to(device)
count = torch.zeros(num_points).to(device)  # 记录每个点被预测的次数
print(model)

RandLANet(
  (fc_start): Linear(in_features=4, out_features=8, bias=True)
  (bn_start): Sequential(
    (0): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (encoder): ModuleList(
    (0): LocalFeatureAggregation(
      (mlp1): SharedMLP(
        (adapter): Identity()
        (conv): Conv1d(8, 8, kernel_size=(1,), stride=(1,))
        (bn): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): LeakyReLU(negative_slope=0.2)
      )
      (lse1): LocalSpatialEncoding(
        (mlp): SharedMLP(
          (adapter): Identity()
          (conv): Conv1d(10, 8, kernel_size=(1,), stride=(1,))
          (bn): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation): ReLU()
        )
      )
      (pool1): AttentivePooling(
        (linear): Linear(in_features=16, out_features=16, bias=True)
        (softmax): Softmax(dim=-2)
    

## 第8步 在没有梯度情况下对点云类别进行预测
- 需要注意的是，predict.yaml文件中的num_epoch、batch_size、step_size、num_points_per_step需要协调好
- 保证点云内的所有点都能预测多次

In [11]:
with torch.no_grad():
    for i in tqdm(range(merged_dict["epoch"])):
        for points in dataloader:
            points = {k: v.to(device, non_blocking=True) for k, v in points.items()}
            feature = points['xyz']
            indices_step = points['index']
            if merged_dict["HasRGB"]:
                rgb = points['rgb']
                feature = torch.cat((feature, rgb), dim=2)
            if merged_dict["HasIntensity"]:
                intensity = points['intensity'].unsqueeze(-1)
                feature = torch.cat((feature, intensity), dim=2)

            # 模型预测 (省略特征提取代码)
            scores = model(feature).to(device).permute(0, 2, 1)
            scores = torch.softmax(scores, dim=-1)
            # 合并预测结果
            global_indices = indices_step.to(device)
            for i in range(merged_dict["batch_size"]):
                point_indices = global_indices[i:i + 1].view(-1).long().to(device)  # 确保为整数类型
                current_scores = scores[i].to(device)  # 确保为浮点类型
                all_preds = all_preds.to(device)
                all_preds[point_indices] += current_scores
                count[point_indices] += 1
# 平均投票结果
all_preds /= count.clamp(min=1).unsqueeze(-1)
all_preds = all_preds.cpu()
labels = torch.argmax(all_preds, dim=-1)
print(labels)

100%|██████████| 10/10 [02:46<00:00, 16.70s/it]

tensor([1, 1, 1,  ..., 1, 1, 1])


## 第9步 读取完整原始点云，将预测的label嵌入点云信息并输出

In [12]:
red = np.zeros(pos.shape)
green = np.zeros(pos.shape)
blue = np.zeros(pos.shape)
if merged_dict["PredictHasRGB"]:
    red = point_r
    green = point_g
    blue = point_b

lasfilename = 'EDITED.las'
output_path = os.path.join(merged_dict["save_path"], lasfilename)
output_path = output_path.replace(os.sep, '/')

# 创建las文件
# 1. Create a new header
header = laspy.LasHeader(point_format=3, version="1.2")
# header.offsets = np.min(cluster_points, axis=0)
# header.scales = np.array([1, 1, 1])

# 2. Create a Las
las = laspy.LasData(header)

las.x = pos[:, 0]
las.y = pos[:, 1]
las.z = pos[:, 2]

las.red = red
las.green = green
las.blue = blue

las.classification = labels.numpy()

las.write(output_path)
print(las.x.shape)

(1277925,)
